In [1]:
import pandas as pd
import numpy as np
import csv as csv
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [2]:
# Data cleanup
# TRAIN DATA
os.chdir(os.path.expanduser("~/Dropbox/Projects/data/kaggle/leaf_classification/"))
train = pd.read_csv('train.csv', header=0)        # Load the train file into a dataframe
test = pd.read_csv('test.csv', header=0)
train.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [3]:
con_all = train.select_dtypes(include=['float64', 'int64']).columns.values
outcome = np.array(['id'])
con = np.setdiff1d(con_all, outcome)
con

array(['margin1', 'margin10', 'margin11', 'margin12', 'margin13',
       'margin14', 'margin15', 'margin16', 'margin17', 'margin18',
       'margin19', 'margin2', 'margin20', 'margin21', 'margin22',
       'margin23', 'margin24', 'margin25', 'margin26', 'margin27',
       'margin28', 'margin29', 'margin3', 'margin30', 'margin31',
       'margin32', 'margin33', 'margin34', 'margin35', 'margin36',
       'margin37', 'margin38', 'margin39', 'margin4', 'margin40',
       'margin41', 'margin42', 'margin43', 'margin44', 'margin45',
       'margin46', 'margin47', 'margin48', 'margin49', 'margin5',
       'margin50', 'margin51', 'margin52', 'margin53', 'margin54',
       'margin55', 'margin56', 'margin57', 'margin58', 'margin59',
       'margin6', 'margin60', 'margin61', 'margin62', 'margin63',
       'margin64', 'margin7', 'margin8', 'margin9', 'shape1', 'shape10',
       'shape11', 'shape12', 'shape13', 'shape14', 'shape15', 'shape16',
       'shape17', 'shape18', 'shape19', 'shape2', 'shape

In [4]:
cat_all = train.select_dtypes(include=['object']).columns.values
outcome = np.array(['species'])
cat = np.setdiff1d(cat_all, outcome)
cat

array([], dtype=object)

In [9]:
X_train = train[con]
X_test = test[con]

In [6]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
Y_train = lb.fit_transform(train.species)
Y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
X_train.shape

(990, 192)

In [21]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
output = OneVsRestClassifier(SVC(kernel='linear', probability=True)).fit(X_train, Y_train).predict_proba(X_test)
# output

outdf = pd.DataFrame(output)
outdf.columns = list(lb.classes_)
outdf['id'] = test['id']
outdf = outdf[["id"] + list(lb.classes_)]
# outdf.head()
outdf.to_csv('submissions/onevstherest_svc.csv', index=False)
print 'Done.'

array([[  6.39942025e-06,   5.04600502e-05,   2.80127092e-06, ...,
          1.98623872e-06,   1.65760244e-05,   2.55158447e-06],
       [  1.81937042e-06,   4.68915123e-05,   1.50228196e-05, ...,
          7.83717594e-04,   1.20363327e-05,   2.65066911e-04],
       [  1.23150253e-04,   9.39651399e-01,   4.46367360e-04, ...,
          7.79670084e-07,   3.30562215e-06,   1.98938876e-02],
       ..., 
       [  9.04642995e-05,   3.75246688e-04,   3.78797081e-04, ...,
          2.08660987e-05,   8.72422078e-06,   6.34340418e-06],
       [  5.46947566e-05,   1.87188679e-05,   1.57163550e-04, ...,
          2.49207812e-06,   7.18606774e-05,   7.14303036e-06],
       [  1.00000010e-07,   4.55341673e-04,   3.18176603e-05, ...,
          1.66021643e-05,   1.13284610e-04,   6.47149837e-05]])

In [27]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

output = OneVsRestClassifier(RandomForestClassifier(n_estimators=100)).fit(X_train, Y_train).predict_proba(X_test)
output

outdf = pd.DataFrame(output)
outdf.columns = list(lb.classes_)
outdf['id'] = test['id']
outdf = outdf[["id"] + list(lb.classes_)]
# outdf.head()
outdf.to_csv('submissions/onevstherest_rf.csv', index=False)
print 'Done.'

Done.


In [29]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier

output = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100)).fit(X_train, Y_train).predict_proba(X_test)
output

array([[  3.05223731e-05,   2.20754706e-05,   3.02519463e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.72510848e-05],
       [  3.05223731e-05,   2.20754706e-05,   5.51688981e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.72510848e-05],
       [  3.05223731e-05,   8.96064256e-05,   3.02519463e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.87134347e-05],
       ..., 
       [  3.05223731e-05,   2.20754706e-05,   3.02519463e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.72510848e-05],
       [  3.05223731e-05,   2.20754706e-05,   3.02519463e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.72510848e-05],
       [  3.05223731e-05,   3.99921180e-05,   3.02519463e-05, ...,
          2.66953210e-05,   2.69556842e-05,   2.72510848e-05]])

In [30]:
outdf = pd.DataFrame(output)
outdf.columns = list(lb.classes_)
outdf['id'] = test['id']
outdf = outdf[["id"] + list(lb.classes_)]
# outdf.head()
outdf.to_csv('submissions/onevstherest_gbm.csv', index=False)
print 'Done.'

Done.
